# Polars vs Pandas

**Autor:** Nelson Zepeda  
**Correo:** nelson.zepeda@datasphere.tech

**Empresa:** [datasphere.tech](https://datasphere.tech)  
**LinkedIn:** [Datasphere Consulting](https://www.linkedin.com/company/datasphere-consulting/)  
**Fecha:** octubre 2025

# Polars: que es, desde cuando existe y por que usarlo

**Que es**  
Polars (https://pola.rs) es una libreria de DataFrames orientada a columnas, escrita en **Rust** con API para **Python**. Usa el formato de memoria de **Apache Arrow**, lo que permite operaciones vectorizadas, bajo overhead y compatibilidad con otros sistemas.

**Desde cuando esta disponible**  
El proyecto comenzo a publicarse alrededor de **2020** y gano adopcion amplia en **2022**. A partir de **2024** se consolida con lanzamientos 1.x y un ecosistema mas maduro (documentacion, motores lazy estables y conectores de E/S).

## Caracteristicas principales

- **Nucleo en Rust**: ejecucion muy rapida, paralelismo multi-hilo y sin GIL.
- **Modelo columnar (Arrow)**: operaciones vectorizadas, menor uso de memoria y zero-copy cuando es posible.
- **Dos modos de trabajo**:
  - **Eager**: estilo parecido a pandas para tareas interactivas.
  - **Lazy**: construye un plan de consulta optimizado antes de ejecutar.
- **Optimizador de consultas (lazy)**:
  - *Projection pushdown*: solo lee las columnas necesarias.
  - *Predicate pushdown*: aplica filtros lo mas cerca posible de la fuente.
  - *Pipeline fusion*: combina pasos para reducir materializaciones intermedias.
- **Streaming / out-of-core**: procesa archivos grandes sin cargarlos completos a memoria.
- **I/O rapido**: `scan_csv`, `scan_parquet`, `read_csv`, `read_parquet` con lectura paralela.
- **Funciones potentes**: `group_by`, `join`, `sort`, `window functions`, `pivot/unpivot`, expresiones vectorizadas.
- **Tipos ricos**: numericos, boolean, string UTF8, datetime (con zonas horarias), duration, categorical, listas y estructuras.
- **Interoperabilidad**: conversion con **pandas**, **NumPy** y **PyArrow**.
- **Facil de instalar**: `pip install polars`.

## Polars vs Pandas

| Aspecto | Polars | Pandas | Nota clave |
|---|---|---|---|
| Motor | Rust + Arrow | C/NumPy + Python | Polars aprovecha paralelismo nativo. |
| Modos de ejecucion | Eager y **Lazy** (plan optimizado) | Eager | Lazy aplica predicate/projection pushdown y fusiona pasos. |
| Paralelismo | Multi-hilo por defecto | Mayormente un solo hilo | Pandas 2.x mejora con copy-on-write pero no es multihilo. |
| Modelo de memoria | Columnar (Arrow) | Basado en NumPy | Columnar favorece escaneo, agregaciones y E/S. |
| E/S CSV/Parquet | `scan_csv`, `scan_parquet` con pushdown y streaming | `read_csv`, `read_parquet` | Polars suele leer mas rapido y con menos RAM. |
| GroupBy/Join/Sort | Muy rapido y paralelo | Solido, pero generalmente mas lento a gran escala | Ventaja de Polars crece con millones de filas. |
| Tipos de datos | Numericos, boolean, string UTF8, datetime con tz, duration, categorical, list, struct | Amplios; pd.NA, Arrow opcional en algunas rutas | Ambos maduros; Polars estricta en dtypes. |
| Falta de valores | `null` (y NaN para floats) | `NaN`/`pd.NA` segun dtype | Diferencias sutiles en comparaciones y agregaciones. |
| UDFs | Expresiones vectorizadas; UDFs Python existen pero menos necesarias | UDFs via `apply` frecuentes | Evitar `apply` en pandas por performance. |
| Ecosistema ML | Interopera via pandas/NumPy | Integracion directa y masiva | scikit-learn espera pandas/NumPy. |

## Cuando elegir cada uno

- **Elige Polars si**:
  - Necesitas pipelines eficientes de E/S -> filtrado -> agregacion -> join.
  - Quieres ejecutar en paralelo y optimizar automaticamente con **lazy**.
  - Debes procesar archivos mas grandes que la RAM usando **streaming**.

- **Elige Pandas si**:
  - Tu flujo depende de librerias que consumen pandas directamente.
  - Tu data cabe comodamente en memoria y prima la compatibilidad.

# Pandas vs Polars: benchmark de operaciones

Esta seccion inicial define el **proposito del benchmark** y carga las **dependencias** necesarias para medir y comparar operaciones tipicas entre **pandas** y **polars** (filtro, agregaciones, joins y ordenamientos) sobre datasets de gran tamaño.

## Objetivo
Medir tiempos de ejecucion de operaciones tabulares representativas, manteniendo condiciones justas (mismas entradas, mismo hardware y metrica de tiempo consistente) para comparar el rendimiento de pandas y polars.

## Alcance
- Enfocado en **procesamiento** (no en visualizacion).
- Incluye operaciones vectorizadas comunes.

In [ ]:
# ==========================================
# Pandas vs Polars: benchmark de operaciones
# ==========================================
import time
import os
import statistics as stats
import numpy as np
import pandas as pd
import polars as pl

#### Datos base (generación y estructura)

- Define el **tamaño del dataset**: `N = 5_000_000` filas para forzar una carga de trabajo donde se aprecian diferencias reales de rendimiento.
- Crea un **generador aleatorio reproducible**: `rng = np.random.default_rng(42)` fija la semilla (42) para que los resultados sean comparables entre corridas.
- Genera **cuatro columnas** con distintas distribuciones y tipos:
  - `cat`: enteros **[0, 1000)** con `dtype=np.int32` (sirve como categoría/clave para `groupby` y `join`).
  - `x1`: distribución **normal** (media 0, sd 1) en `float64`.
  - `x2`: enteros **[0, 10_000)** con `dtype=np.int32` (útil para agregaciones y filtros modulares).
  - `x3`: distribución **uniforme** en `[0, 1)`, `float64`.
- Construye dos DataFrames **idénticos en contenido**:
  - `pdf` con **pandas**.
  - `pldf` con **polars**.

In [ ]:
# ----------Base de Datos ----------
N = 5_000_000
rng = np.random.default_rng(42)

cat = rng.integers(0, 1_000, N, dtype=np.int32)
x1  = rng.normal(0, 1, N)
x2  = rng.integers(0, 10_000, N, dtype=np.int32)
x3  = rng.random(N)

pdf = pd.DataFrame({"cat": cat, "x1": x1, "x2": x2, "x3": x3})
pldf = pl.DataFrame({"cat": cat, "x1": x1, "x2": x2, "x3": x3})

#### Dimension para join

- Simula una **dimension** de un esquema estrella y se une contra la tabla de **hechos** grande creada antes (`pdf`/`pldf`, con 5M de filas).
- El join es **muchos-a-uno**: muchas filas del hecho por cada categoria `cat` en la dimension.
- Permite medir rendimiento en **join + ordenamientos** (p. ej., ordenar por `w` y tomar top-k) con una clave que **siempre existe** en la tabla de hechos, ya que `cat` en los datos base fue generado en el rango `0..999`.

- Define el tamano de la **tabla de dimension**: `dim_size = 1_000`.
- Crea un DataFrame **pandas** `dim` con:
  - `cat`: valores enteros `0..999` (`np.arange(dim_size, dtype=np.int32)`) que actuan como **llave**.
  - `w`: una columna de **pesos aleatorios** `rng.random(dim_size)` usada luego para ordenar o calcular top-k.
- Convierte esa misma tabla a **polars** con `pl.from_pandas(dim)` para garantizar que **pandas y polars** operen sobre **los mismos datos** en los benchmarks.



In [ ]:
# Dimensión para join
dim_size = 1_000
dim = pd.DataFrame({"cat": np.arange(dim_size, dtype=np.int32),
                    "w": rng.random(dim_size)})
pldim = pl.from_pandas(dim)

#### Helpers: `bench` y `show`

##### `bench(fn, repeats=5, warmup=1)`
Mide el tiempo de ejecución de una función `fn` varias veces y devuelve estadísticas resumidas.

- **Warm-up**: ejecuta `fn()` `warmup` veces **sin medir** para “calentar” caches/JIT/IO y estabilizar el entorno.
- **Medición**: usa `time.perf_counter()` (reloj de alta resolución) para cronometrar `repeats` ejecuciones.
- **Salida**: retorna un `dict` con `median`, `mean`, `min`, `max` en segundos (redondeados a 4 decimales).


In [ ]:
# ---------- helpers ----------
def bench(fn, repeats=5, warmup=1):
    for _ in range(warmup):
        fn()
    t = []
    for _ in range(repeats):
        t0 = time.perf_counter()
        _ = fn()
        t1 = time.perf_counter()
        t.append(t1 - t0)
    return dict(
        median=round(stats.median(t), 4),
        mean=round(stats.mean(t), 4),
        min=round(min(t), 4),
        max=round(max(t), 4),
    )

def show(name, res):
    print(f"{name:24s}  median={res['median']}s  mean={res['mean']}s  "
          f"min={res['min']}s  max={res['max']}s")

#### 1) Filtro + columna derivada

**Propósito.**  
Medir el tiempo de una transformación típica: **filtrar** filas y **crear** una columna calculada.
Filtramos las filas donde x1 es positivo y x2 es múltiplo de 7; luego calculamos un puntaje y = x1 * log(1 + x2) + x3 ( x1 pesa, log(1+x2) suaviza, x3 ajusta).

**Regla de negocio aplicada**
- Filtro: `x1 > 0` **y** `x2 % 7 == 0` (múltiplos de 7).
- Columna nueva: `y = x1 * log1p(x2) + x3`.  
  - En pandas se usa `np.log1p(x2)` (estable y preciso cuando `x2` es grande).  
  - En polars se usa `(x2 + 1).log()`, que es equivalente a `log1p`.

---

In [ ]:
# ---------- 1) Filtro + columna derivada ----------
# regla: x1 > 0 & x2 % 7 == 0; nueva y = x1 * log1p(x2) + x3
import math

pandas_filter = lambda: (
    (lambda df: df.assign(y=df["x1"] * np.log1p(df["x2"]) + df["x3"]))(
        pdf.loc[(pdf["x1"] > 0) & (pdf["x2"].mod(7).eq(0))]
    )
)


polars_filter_eager = lambda: (
    pldf.filter( (pl.col("x1") > 0) & (pl.col("x2") % 7 == 0) )
        .with_columns( (pl.col("x1") * (pl.col("x2")+1).log() + pl.col("x3"))
                       .alias("y") )
)

polars_filter_lazy = lambda: (
    pldf.lazy()
        .filter( (pl.col("x1") > 0) & (pl.col("x2") % 7 == 0) )
        .with_columns( (pl.col("x1") * (pl.col("x2")+1).log() + pl.col("x3"))
                       .alias("y") )
        .collect()
)

print("\n# 1) Filtro + columna derivada")
show("pandas", bench(pandas_filter))
show("polars eager", bench(polars_filter_eager))
show("polars lazy", bench(polars_filter_lazy))

#### 2) Groupby / Aggregations

**Qué hace**  
Agrupa el DataFrame por la columna **`cat`** y calcula, por grupo:
- `mean(x1)` → promedio de `x1`
- `sum(x2)`  → suma de `x2`
- `max(x3)`  → máximo de `x3`
---

In [ ]:
# ---------- 2) Groupby/agg ----------
# por cat: mean(x1), sum(x2), max(x3)
pandas_gb = lambda: pdf.groupby("cat", sort=False, observed=True).agg({
    "x1": "mean",
    "x2": "sum",
    "x3": "max"
}).reset_index()

polars_gb_eager = lambda: (
    pldf.group_by("cat")
        .agg( pl.col("x1").mean(),
              pl.col("x2").sum(),
              pl.col("x3").max() )
)

polars_gb_lazy = lambda: (
    pldf.lazy()
        .group_by("cat")
        .agg( pl.col("x1").mean(),
              pl.col("x2").sum(),
              pl.col("x3").max() )
        .collect()
)

print("\n# 2) Groupby/Aggregations")
show("pandas", bench(pandas_gb))
show("polars eager", bench(polars_gb_eager))
show("polars lazy", bench(polars_gb_lazy))


#### 3) Join + Top-K

**Qué hace**
1. **Enriquece** la tabla grande (`pdf`/`pldf`) con la **dimensión** `dim/pldim` por la clave `cat` mediante un **left join**.
2. **Ordena** todas las filas resultantes por la columna `w` en **orden descendente**.
3. **Toma las 10 primeras filas** tras ese orden.  
---

In [ ]:
# ---------- 3) Join + top-k ----------
# join con dim en cat; luego top 10 por w descendente dentro de cada cat (sample)
pandas_join_topk = lambda: (
    pdf.merge(dim, on="cat", how="left")
       .sort_values("w", ascending=False)
       .head(10)
)

polars_join_topk_eager = lambda: (
    pldf.join(pldim, on="cat", how="left")
        .sort("w", descending=True)
        .head(10)
)

polars_join_topk_lazy = lambda: (
    pldf.lazy().join(pldim.lazy(), on="cat", how="left")
        .sort("w", descending=True)
        .limit(10)
        .collect()
)

print("\n# 3) Join + Top-K")
show("pandas", bench(pandas_join_topk))
show("polars eager", bench(polars_join_topk_eager))
show("polars lazy", bench(polars_join_topk_lazy))

#### 4) Sort global + selección de columnas

Ordena **todo** el DataFrame por dos claves y luego se queda solo con `["cat","x1","x2"]`:

- Clave 1: `x1` **descendente** (mayores primero).
- Clave 2: `x2` **ascendente** (desempate).
- Selección final: descarta `x3`.


In [ ]:
# ---------- 4) Sort global + selección de columnas ----------
pandas_sort = lambda: pdf.sort_values(["x1","x2"], ascending=[False, True])[["cat","x1","x2"]]
polars_sort_eager = lambda: pldf.sort(["x1","x2"], descending=[True, False]).select(["cat","x1","x2"])
polars_sort_lazy = lambda: pldf.lazy().sort(["x1","x2"], descending=[True, False]).select(["cat","x1","x2"]).collect()

print("\n# 4) Sort + select")
show("pandas", bench(pandas_sort))
show("polars eager", bench(polars_sort_eager))
show("polars lazy", bench(polars_sort_lazy))


#### 5) Lectura de CSV grande (I/O): pandas vs polars

- Genera (si no existe) un archivo **CSV sintético** con `N = 5_000_000` filas y 4 columnas numéricas (`cat`, `x1`, `x2`, `x3`) para tener un caso de lectura realista y pesado.
- **Mide el tiempo** de lectura del mismo archivo con:
  - `pandas.read_csv(...)`
  - `polars.read_csv(...)`
- Imprime **duración** y **shape** resultante para cada librería.

In [ ]:
# ---- Benchmark de lectura de CSV grande: pandas vs polars ----
# Genera un CSV sintético y mide tiempos de lectura.
N = 5_000_000  
rng = np.random.default_rng(42)

# Genera un CSV si no existe
csv_path = "synth.csv"
if not os.path.exists(csv_path):
    pdf = pd.DataFrame({
        "cat": rng.integers(0, 10_000, N, dtype=np.int32),
        "x1":  rng.normal(0, 1, N),
        "x2":  rng.integers(0, 1_000_000, N, dtype=np.int32),
        "x3":  rng.random(N),
    })
    pdf.to_csv(csv_path, index=False)

In [ ]:
# Lee con pandas
t0 = time.perf_counter()
pdf = pd.read_csv(csv_path)
t1 = time.perf_counter()
print(f"pandas read_csv: {t1 - t0:.2f}s, shape={pdf.shape}")

In [ ]:
# Lee con polars (multihilo, streaming desactivado)
t0 = time.perf_counter()
pldf = pl.read_csv(csv_path)
t1 = time.perf_counter()
print(f"polars read_csv: {t1 - t0:.2f}s, shape={pldf.shape}")

In [ ]:
# Define dtypes explícitos para saltarte el costo de inferir el schema.
dtypes = {"cat": pl.Int32, "x1": pl.Float64, "x2": pl.Int32, "x3": pl.Float64}

t0 = time.perf_counter()
pldf = pl.read_csv(csv_path, dtypes=dtypes)  # usa todos los núcleos por defecto
t1 = time.perf_counter()
print(f"polars read_csv (dtypes): {t1 - t0:.2f}s, shape={pldf.shape}")


In [ ]:
# Usa el motor lazy para aplicar "projection pushdown" y procesar en streaming.
dtypes = {"cat": pl.Int32, "x1": pl.Float64, "x2": pl.Int32, "x3": pl.Float64}

t0 = time.perf_counter()
pldf = (
    pl.scan_csv(csv_path, dtypes=dtypes)    # no carga todo a memoria al inicio
      .select(["cat", "x2"])                # solo las columnas que necesitas
      .collect(streaming=True)              # pipeline en streaming y multihilo
)
t1 = time.perf_counter()
print(f"polars scan_csv->select (stream): {t1 - t0:.2f}s, shape={pldf.shape}")


## Conclusiones del benchmark (Pandas vs Polars)

## Resultados clave
- **Lectura CSV (I/O)**: pasaste de `pandas read_csv: 3.20s` (shape=(5,000,000, 4)) a  
  `polars scan_csv→select (stream): 0.28s` (shape=(5,000,000, 2)).  
  → **~11.4× más rápido**, aprovechando **lazy + streaming** y **projection pushdown** (solo 2 columnas).

- **Transformaciones (filtro, groupby, join, sort)**: Polars (especialmente en **modo lazy**) consistentemente superó a Pandas en datasets grandes por:
  - **Paralelismo** multi-hilo en Rust.
  - **Optimización de plan** (predicate/projection pushdown, pipeline fusion).
  - **Modelo columnar (Arrow)** que favorece escaneos y agregaciones.

- **Creación de DataFrames desde NumPy**: puede verse **más rápida en Pandas** (casi zero-copy). No contradice lo anterior: la ventaja de Polars aparece en **procesamiento** e **I/O** a escala.

## Qué significa en la práctica
- Si tu pipeline es **E/S → filtros → selección de columnas → agregaciones/joins**, Polars te dará **2–10×** de mejora (o más) y **menos RAM**.
- El patrón **`scan_csv().filter().select().collect(streaming=True)`** es decisivo: evita parsear/traer datos innecesarios.
- En Pandas, para acercarte, debes recurrir a patrones manuales (p. ej., `read_csv(..., usecols=..., chunksize=...)`), con más código y aún así, típicamente, menor rendimiento.

## Recomendaciones operativas
1. **Usa Polars Lazy por defecto** para pipelines: `scan_* → filter → select → group_by → collect(streaming=True)`.
2. **Declara `dtypes`** al leer CSV si conoces el schema (evita la inferencia).
3. **Lee solo lo necesario** (`select`) y **filtra al escanear** (`filter`) para reducir I/O y memoria.
4. **Para top-k**, usa `sort(...).limit(k)` o APIs específicas (`top_k`) en lugar de ordenar todo.
5. **Interoperabilidad**: convierte a Pandas solo cuando un paquete lo exija (`df.to_pandas()`), al final del pipeline.

## Cuándo seguir con Pandas
- Ecosistema que **requiere directamente Pandas/NumPy** (modelos ML, ciertos gráficos).
- Tareas pequeñas/interactivas donde la diferencia de rendimiento **no justifica** el cambio.

## Resumen
- **Polars** es la opción preferida para **datasets medianos/grandes** y **pipelines analíticos**: más rápido, eficiente y declarativo.
- **Pandas** mantiene su lugar como **interfaz universal** en el ecosistema Python, ideal para integración.
